In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

In [2]:
!ls /home/chec/data/bengali

ls: cannot access '/home/chec/data/bengali': No such file or directory


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/mnt/chicm/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236
#SIZE = 128

In [8]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #img = crop_resize(img, pad=0)
        #print(img.shape)
        #if self.train_mode:
        #    augs = get_train_augs()
        #    img = augs(image=img)['image']

        #img  = cv2.resize(img, (SIZE, SIZE))
        
        img = np.expand_dims(img, axis=-1)
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        MEAN = [ 0.06922848809290576 ]
        STD = [ 0.20515700083327537 ]
        
        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

# model

In [12]:
#import pretrainedmodels

In [13]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [16]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

In [17]:
#model.last_linear.in_features

In [18]:
class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        #self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        #self.fix_input_layer()
        
    def fix_input_layer(self):
        if self.backbone_name in ['se_resnext50_32x4d', 'se_resnext101_32x4d', 'se_resnet50', 'senet154', 'se_resnet152', 'nasnetmobile', 'mobilenet', 'nasnetalarge']:
            #self.backbone = eval(backbone_name)()
            #print(self.backbone.layer0.conv1)
            w = self.backbone.layer0.conv1.weight.data
            self.backbone.layer0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            #self.backbone.layer0.conv1.weight = torch.nn.Parameter(torch.cat((w, w[:, 2, :, :].unsqueeze(1)), dim=1))
            self.backbone.layer0.conv1.weight = torch.nn.Parameter(w[:, 0, :, :].unsqueeze(1))
        
    def logits(self, x):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [19]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [20]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [21]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [22]:
round(1/9, 6)

0.111111

In [23]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [24]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [25]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [26]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [27]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [28]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [29]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [30]:
np.random.random()

0.936591631689417

In [31]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam

In [32]:
def train(args):
    global model

    if args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
    elif args.optim == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    elif args.optim == 'Over9000':
        optimizer = Over9000(model.parameters(), lr=args.lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-5)

    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.t_max, eta_min=args.min_lr)
        
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    #if args.lrs == 'plateau':
    #    lr_scheduler.step(best_metrics)
    #else:
    #    lr_scheduler.step()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #do_mixup = False #(np.random.random() < 0.4)
            
            #if do_mixup:
            #    img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            
            
            #if do_mixup:
            #    loss = mixup_criterion(outputs, targets)
            #else:
            #    loss = criterion(outputs, targets)
            r = np.random.rand()
            #if args.beta > 0 and r < args.cutmix_prob:
            if r < 0.5:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs = model(img)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
                img, targets = mixup(img, targets)
                outputs = model(img)
                loss = mixup_criterion(outputs, targets)
                #loss = criterion(outputs, targets)
            
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            
            #loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [33]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'model3_se_resnext50_fold2_mixup_cutmix.pth'
args.predict = False
args.optim = 'Adam'
args.lr = 3e-5
args.lrs = 'cosine'
args.t_max = 12
args.factor = 0.1
args.patience = 0
args.min_lr = 2e-6
args.iter_val = 200
args.num_epochs = 100000
args.batch_size = 1024
args.val_batch_size = 1024

args.beta = 1.0
args.cutmix_prob = 0.5

In [34]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=2)

(200840, 5)
(200840, 32332)
(160678, 5) (40162, 5)


In [35]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


model file: ./models/se_resnext50_32x4d/model3_se_resnext50_fold2_mixup_cutmix.pth, exist: True
loading ./models/se_resnext50_32x4d/model3_se_resnext50_fold2_mixup_cutmix.pth...


In [36]:
train(args)

{'recall': 0.984254, 'recall_grapheme': 0.976814, 'recall_vowel': 0.991676, 'recall_consonant': 0.99171, 'acc_grapheme': 0.974528, 'acc_vowel': 0.992978, 'acc_consonant': 0.991435, 'loss_grapheme': 0.214347, 'loss_vowel': 0.149057, 'loss_consonant': 0.108816}
    1 | 0.000030 | 045056/160678 | 1.4220 | 1.8856 |
val: {'recall': 0.983727, 'recall_grapheme': 0.976074, 'recall_vowel': 0.991217, 'recall_consonant': 0.991544, 'acc_grapheme': 0.974603, 'acc_vowel': 0.992804, 'acc_consonant': 0.991186, 'loss_grapheme': 0.203053, 'loss_vowel': 0.151237, 'loss_consonant': 0.10703}


/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    2 | 0.000030 | 090112/160678 | 1.7320 | 2.2550 |
val: {'recall': 0.983109, 'recall_grapheme': 0.975741, 'recall_vowel': 0.990863, 'recall_consonant': 0.99009, 'acc_grapheme': 0.974578, 'acc_vowel': 0.992655, 'acc_consonant': 0.99126, 'loss_grapheme': 0.19116, 'loss_vowel': 0.146013, 'loss_consonant': 0.104978}
    3 | 0.000028 | 135168/160678 | 1.5671 | 2.2238 |
val: {'recall': 0.983459, 'recall_grapheme': 0.975641, 'recall_vowel': 0.991604, 'recall_consonant': 0.990948, 'acc_grapheme': 0.97413, 'acc_vowel': 0.993103, 'acc_consonant': 0.991136, 'loss_grapheme': 0.218874, 'loss_vowel': 0.165622, 'loss_consonant': 0.122124}
    5 | 0.000026 | 020480/160678 | 1.2641 | 2.2607 |
val: {'recall': 0.983229, 'recall_grapheme': 0.976018, 'recall_vowel': 0.991318, 'recall_consonant': 0.989561, 'acc_grapheme': 0.974354, 'acc_vowel': 0.993028, 'acc_consonant': 0.99131, 'loss_grapheme': 0.214015, 'loss_vowel': 0.158783, 'loss_consonant': 0.110964}
    6 | 0.000023 | 065536/160678 | 0.3876 | 2.08

   35 | 0.000026 | 143360/160678 | 3.6425 | 2.2030 |
val: {'recall': 0.983605, 'recall_grapheme': 0.975908, 'recall_vowel': 0.991186, 'recall_consonant': 0.99142, 'acc_grapheme': 0.974453, 'acc_vowel': 0.993003, 'acc_consonant': 0.991484, 'loss_grapheme': 0.22006, 'loss_vowel': 0.155876, 'loss_consonant': 0.115851}
   37 | 0.000023 | 028672/160678 | 3.6594 | 2.4070 |
val: {'recall': 0.982949, 'recall_grapheme': 0.974992, 'recall_vowel': 0.991159, 'recall_consonant': 0.990651, 'acc_grapheme': 0.974204, 'acc_vowel': 0.993153, 'acc_consonant': 0.99141, 'loss_grapheme': 0.249065, 'loss_vowel': 0.197817, 'loss_consonant': 0.135849}
   38 | 0.000020 | 073728/160678 | 3.2654 | 2.1153 |
val: {'recall': 0.982934, 'recall_grapheme': 0.975593, 'recall_vowel': 0.990523, 'recall_consonant': 0.990027, 'acc_grapheme': 0.974254, 'acc_vowel': 0.992729, 'acc_consonant': 0.991484, 'loss_grapheme': 0.210753, 'loss_vowel': 0.164874, 'loss_consonant': 0.113794}
   39 | 0.000016 | 118784/160678 | 2.4828 | 2.

KeyboardInterrupt: 

In [36]:
train(args)

{'recall': 0.983334, 'recall_grapheme': 0.97579, 'recall_vowel': 0.990465, 'recall_consonant': 0.991293, 'acc_grapheme': 0.973184, 'acc_vowel': 0.991933, 'acc_consonant': 0.99009, 'loss_grapheme': 0.25881, 'loss_vowel': 0.200453, 'loss_consonant': 0.133431}
    1 | 0.000040 | 045056/160678 | 3.6400 | 2.5026 |
val: {'recall': 0.98278, 'recall_grapheme': 0.975052, 'recall_vowel': 0.989611, 'recall_consonant': 0.991407, 'acc_grapheme': 0.972711, 'acc_vowel': 0.991659, 'acc_consonant': 0.990165, 'loss_grapheme': 0.292719, 'loss_vowel': 0.242861, 'loss_consonant': 0.150753}


/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    2 | 0.000039 | 090112/160678 | 4.0569 | 2.3631 |
val: {'recall': 0.982796, 'recall_grapheme': 0.974925, 'recall_vowel': 0.989774, 'recall_consonant': 0.991559, 'acc_grapheme': 0.972636, 'acc_vowel': 0.991833, 'acc_consonant': 0.989667, 'loss_grapheme': 0.287815, 'loss_vowel': 0.252474, 'loss_consonant': 0.154994}
    3 | 0.000037 | 135168/160678 | 2.2190 | 2.3725 |
val: {'recall': 0.982799, 'recall_grapheme': 0.975084, 'recall_vowel': 0.990086, 'recall_consonant': 0.990941, 'acc_grapheme': 0.973009, 'acc_vowel': 0.991982, 'acc_consonant': 0.99014, 'loss_grapheme': 0.220253, 'loss_vowel': 0.169415, 'loss_consonant': 0.112443}
    5 | 0.000034 | 020480/160678 | 2.2770 | 2.5581 |
val: {'recall': 0.983016, 'recall_grapheme': 0.975674, 'recall_vowel': 0.989542, 'recall_consonant': 0.991173, 'acc_grapheme': 0.973009, 'acc_vowel': 0.991758, 'acc_consonant': 0.989891, 'loss_grapheme': 0.294731, 'loss_vowel': 0.244157, 'loss_consonant': 0.148621}
    6 | 0.000030 | 065536/160678 | 1.5115 | 

   35 | 0.000034 | 143360/160678 | 1.5977 | 2.4454 |
val: {'recall': 0.982846, 'recall_grapheme': 0.975443, 'recall_vowel': 0.990105, 'recall_consonant': 0.990392, 'acc_grapheme': 0.973084, 'acc_vowel': 0.992007, 'acc_consonant': 0.990464, 'loss_grapheme': 0.253422, 'loss_vowel': 0.191433, 'loss_consonant': 0.130282}
   37 | 0.000030 | 028672/160678 | 2.2746 | 2.2861 |
val: {'recall': 0.983051, 'recall_grapheme': 0.97526, 'recall_vowel': 0.99015, 'recall_consonant': 0.991537, 'acc_grapheme': 0.973283, 'acc_vowel': 0.992231, 'acc_consonant': 0.990513, 'loss_grapheme': 0.24445, 'loss_vowel': 0.191415, 'loss_consonant': 0.124084}
   38 | 0.000026 | 073728/160678 | 1.2148 | 2.6391 |
val: {'recall': 0.983084, 'recall_grapheme': 0.975452, 'recall_vowel': 0.990035, 'recall_consonant': 0.991398, 'acc_grapheme': 0.973233, 'acc_vowel': 0.992107, 'acc_consonant': 0.990563, 'loss_grapheme': 0.238574, 'loss_vowel': 0.178383, 'loss_consonant': 0.119639}
   39 | 0.000021 | 118784/160678 | 0.2507 | 2.

   69 | 0.000026 | 036864/160678 | 2.2043 | 2.5326 |
val: {'recall': 0.983071, 'recall_grapheme': 0.975765, 'recall_vowel': 0.990299, 'recall_consonant': 0.990457, 'acc_grapheme': 0.973482, 'acc_vowel': 0.992231, 'acc_consonant': 0.990563, 'loss_grapheme': 0.259903, 'loss_vowel': 0.203122, 'loss_consonant': 0.135824}
   70 | 0.000021 | 081920/160678 | 1.2931 | 2.6078 |
val: {'recall': 0.98334, 'recall_grapheme': 0.976002, 'recall_vowel': 0.990586, 'recall_consonant': 0.990772, 'acc_grapheme': 0.973283, 'acc_vowel': 0.992107, 'acc_consonant': 0.990264, 'loss_grapheme': 0.351826, 'loss_vowel': 0.261734, 'loss_consonant': 0.184408}
   71 | 0.000015 | 126976/160678 | 2.1249 | 2.2977 |
val: {'recall': 0.983046, 'recall_grapheme': 0.975813, 'recall_vowel': 0.990283, 'recall_consonant': 0.990276, 'acc_grapheme': 0.973433, 'acc_vowel': 0.992107, 'acc_consonant': 0.990762, 'loss_grapheme': 0.236249, 'loss_vowel': 0.17963, 'loss_consonant': 0.122693}
   73 | 0.000011 | 012288/160678 | 2.3508 | 2

...

  203 | 0.000004 | 135168/160678 | 2.2688 | 2.1419 |
val: {'recall': 0.983428, 'recall_grapheme': 0.975805, 'recall_vowel': 0.990562, 'recall_consonant': 0.991541, 'acc_grapheme': 0.974155, 'acc_vowel': 0.992605, 'acc_consonant': 0.991211, 'loss_grapheme': 0.238234, 'loss_vowel': 0.211634, 'loss_consonant': 0.137286}
  205 | 0.000007 | 020480/160678 | 3.9685 | 2.5105 |
val: {'recall': 0.983705, 'recall_grapheme': 0.976083, 'recall_vowel': 0.991177, 'recall_consonant': 0.991476, 'acc_grapheme': 0.974155, 'acc_vowel': 0.992879, 'acc_consonant': 0.991086, 'loss_grapheme': 0.251153, 'loss_vowel': 0.20111, 'loss_consonant': 0.136639}
  206 | 0.000011 | 065536/160678 | 4.1559 | 2.5022 |
val: {'recall': 0.983343, 'recall_grapheme': 0.976023, 'recall_vowel': 0.990775, 'recall_consonant': 0.99055, 'acc_grapheme': 0.97408, 'acc_vowel': 0.992705, 'acc_consonant': 0.991235, 'loss_grapheme': 0.235226, 'loss_vowel': 0.198388, 'loss_consonant': 0.131999}
  207 | 0.000015 | 110592/160678 | 0.4002 | 2.

  237 | 0.000011 | 028672/160678 | 1.7566 | 2.1513 |
val: {'recall': 0.983431, 'recall_grapheme': 0.976164, 'recall_vowel': 0.990984, 'recall_consonant': 0.99041, 'acc_grapheme': 0.974379, 'acc_vowel': 0.992754, 'acc_consonant': 0.991435, 'loss_grapheme': 0.187703, 'loss_vowel': 0.137314, 'loss_consonant': 0.096427}
  238 | 0.000015 | 073728/160678 | 1.9309 | 2.0376 |
val: {'recall': 0.983448, 'recall_grapheme': 0.976286, 'recall_vowel': 0.990745, 'recall_consonant': 0.990474, 'acc_grapheme': 0.974429, 'acc_vowel': 0.992854, 'acc_consonant': 0.991509, 'loss_grapheme': 0.203835, 'loss_vowel': 0.15776, 'loss_consonant': 0.106257}
  239 | 0.000020 | 118784/160678 | 0.6480 | 2.1948 |
val: {'recall': 0.982903, 'recall_grapheme': 0.975394, 'recall_vowel': 0.990489, 'recall_consonant': 0.990335, 'acc_grapheme': 0.973682, 'acc_vowel': 0.992655, 'acc_consonant': 0.991285, 'loss_grapheme': 0.216014, 'loss_vowel': 0.177446, 'loss_consonant': 0.118972}
  241 | 0.000026 | 004096/160678 | 1.0244 | 1

  270 | 0.000021 | 081920/160678 | 2.0218 | 2.2593 |
val: {'recall': 0.983335, 'recall_grapheme': 0.975043, 'recall_vowel': 0.991614, 'recall_consonant': 0.991639, 'acc_grapheme': 0.974005, 'acc_vowel': 0.993078, 'acc_consonant': 0.991335, 'loss_grapheme': 0.222014, 'loss_vowel': 0.173602, 'loss_consonant': 0.124221}
  271 | 0.000026 | 126976/160678 | 1.7406 | 2.1598 |
val: {'recall': 0.983128, 'recall_grapheme': 0.974962, 'recall_vowel': 0.991236, 'recall_consonant': 0.991352, 'acc_grapheme': 0.974453, 'acc_vowel': 0.992879, 'acc_consonant': 0.991285, 'loss_grapheme': 0.19939, 'loss_vowel': 0.160041, 'loss_consonant': 0.110777}
  273 | 0.000030 | 012288/160678 | 0.7693 | 2.3012 |
val: {'recall': 0.984254, 'recall_grapheme': 0.976814, 'recall_vowel': 0.991676, 'recall_consonant': 0.99171, 'acc_grapheme': 0.974528, 'acc_vowel': 0.992978, 'acc_consonant': 0.991435, 'loss_grapheme': 0.214347, 'loss_vowel': 0.149057, 'loss_consonant': 0.108816}
** saved
  274 | 0.000034 | 057344/160678 | 4

  303 | 0.000030 | 135168/160678 | 0.9878 | 2.2201 |
val: {'recall': 0.983684, 'recall_grapheme': 0.975709, 'recall_vowel': 0.991264, 'recall_consonant': 0.992053, 'acc_grapheme': 0.974379, 'acc_vowel': 0.993078, 'acc_consonant': 0.991385, 'loss_grapheme': 0.240206, 'loss_vowel': 0.184247, 'loss_consonant': 0.133195}
  305 | 0.000034 | 020480/160678 | 1.7379 | 2.2473 |
val: {'recall': 0.982881, 'recall_grapheme': 0.975303, 'recall_vowel': 0.990729, 'recall_consonant': 0.990192, 'acc_grapheme': 0.974354, 'acc_vowel': 0.992655, 'acc_consonant': 0.991435, 'loss_grapheme': 0.207132, 'loss_vowel': 0.165091, 'loss_consonant': 0.114362}
  306 | 0.000037 | 065536/160678 | 1.5356 | 1.9651 |
val: {'recall': 0.983431, 'recall_grapheme': 0.976016, 'recall_vowel': 0.991293, 'recall_consonant': 0.990399, 'acc_grapheme': 0.974503, 'acc_vowel': 0.992954, 'acc_consonant': 0.991385, 'loss_grapheme': 0.192663, 'loss_vowel': 0.134286, 'loss_consonant': 0.096645}
  307 | 0.000039 | 110592/160678 | 0.5308 |

KeyboardInterrupt: 

In [36]:
train(args)

{'recall': 0.051099, 'recall_grapheme': 0.000989, 'recall_vowel': 0.088401, 'recall_consonant': 0.114016, 'acc_grapheme': 0.00371, 'acc_vowel': 0.109706, 'acc_consonant': 0.367138, 'loss_grapheme': 5.189187, 'loss_vowel': 2.427056, 'loss_consonant': 1.819271}


/mnt/chicm/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


    1 | 0.002000 | 045056/160678 | 1.3201 | 3.3993 |
val: {'recall': 0.924224, 'recall_grapheme': 0.907742, 'recall_vowel': 0.940068, 'recall_consonant': 0.941345, 'acc_grapheme': 0.879164, 'acc_vowel': 0.939271, 'acc_consonant': 0.954086, 'loss_grapheme': 0.769752, 'loss_vowel': 0.467442, 'loss_consonant': 0.276683}
** saved
    2 | 0.002000 | 090112/160678 | 1.4723 | 3.8490 |
val: {'recall': 0.946836, 'recall_grapheme': 0.922803, 'recall_vowel': 0.971076, 'recall_consonant': 0.970662, 'acc_grapheme': 0.90613, 'acc_vowel': 0.962925, 'acc_consonant': 0.960112, 'loss_grapheme': 1.044443, 'loss_vowel': 0.359403, 'loss_consonant': 0.220015}
** saved
    3 | 0.002000 | 135168/160678 | 4.2459 | 3.7544 |
val: {'recall': 0.949923, 'recall_grapheme': 0.926272, 'recall_vowel': 0.970635, 'recall_consonant': 0.976511, 'acc_grapheme': 0.909641, 'acc_vowel': 0.952243, 'acc_consonant': 0.93815, 'loss_grapheme': 0.689081, 'loss_vowel': 0.414539, 'loss_consonant': 0.269692}
** saved
    5 | 0.002000 |

   35 | 0.000010 | 159744/160678 | 0.7214 | 2.2880 |

KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)

In [ ]:
del model